## 线性回归，使用框架实现

In [3]:
import numpy as np
import torch
from torch.utils import data

### 1.生成数据

In [2]:
def synthetic_data(w, b, num_examples):
    '''生成 y = Xw + b + 高斯噪声'''
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    print(y.shape)
    y += torch.normal(0, 0.01, y.shape)
    print(y.shape)
    return X, y.reshape((-1, 1))                # matmul后，降维了，需要reshape成二维的，方便后面算loss

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

torch.Size([1000])
torch.Size([1000])


### 2.读取数据
调用API构建dataset和dataloader

In [3]:
def load_array(data_arrays, batch_size, is_train=True):
    '''利用data.TensorDataset构建dataset，is_train代表是否为训练集，是的话需要shuffle'''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[-0.8449, -0.1043],
         [-2.3458, -0.0473],
         [-0.4805,  0.2353],
         [ 1.1556,  0.4102],
         [-0.5372, -0.3888],
         [ 1.5808, -1.6032],
         [-0.8706, -0.7841],
         [-1.0483, -0.7381],
         [-0.7452,  0.4070],
         [-1.8241, -0.4992]]),
 tensor([[ 2.8692],
         [-0.3414],
         [ 2.4200],
         [ 5.1261],
         [ 4.4455],
         [12.8027],
         [ 5.1303],
         [ 4.5956],
         [ 1.3274],
         [ 2.2554]])]

### 3.模型定义

使用框架预定义好的顺序模型和线性层

In [5]:
from torch import nn

net = nn.Sequential(nn.Linear(2,1))

初始化模型参数：调API

In [6]:
net[0].weight.data.normal_(0, 0.01) # 正态分布
net[0].weight.data.fill_(0)         # 全0

tensor([[0., 0.]])

均方差，平方L2范数

In [7]:
loss = nn.MSELoss()

优化方法：实例化SGD

In [8]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

### 4.训练

In [9]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()     # 梯度置零
        l.backward()            # 反向传播算梯度
        trainer.step()          # 优化器更新模型参数
    with torch.no_grad():
        l = loss(net(features), labels)
        print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000221
epoch 2, loss 0.000105
epoch 3, loss 0.000105


### 5.小结

  - 在PyTorch中，data模块提供了数据处理工具，nn模块定义了大量的神经网络层和常见损失函数。
  - 我们可以通过_结尾的方法将参数替换，从而初始化参数。

### 6.练习

Huber's Robust Loss：
$$
l(y, y^{'}) = 
\begin{cases}
|y - y^{'}| - \frac{1}{2} & & if |y - y^{'}|>1 \\
\frac{1}{2}(y - y^{'})^2 & & otherwise
\end{cases}
$$
用L2 Loss待替了L1 Loss的不可导点；

研究损失函数的特性：
- 函数值
- 函数的导数值
- 似然函数：联合概率密度函数的乘积

In [4]:
x = np.linspace(-5, 5, 1000)
x = torch.from_numpy(x)